In [ ]:
from keras.datasets.cifar10 import load_data
import matplotlib.pyplot as plt

(xtrain ,ytrain) ,(xtest ,ytest) = load_data()


In [ ]:
for i in range(100):
  plt.subplot(10 ,10 ,1+i)
  plt.axis('off')
  plt.imshow(xtrain[i])
plt.show()

In [ ]:
print(xtrain.shape)

In [ ]:
# defining Descriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense ,Conv2D ,Flatten ,Dropout ,LeakyReLU
from keras.utils import plot_model
import numpy as np

In [ ]:
def define_descriminator(in_shape=(32,32,3)):

  model = Sequential([

      Conv2D(filters=64 ,kernel_size=(3,3) ,padding='same' ,input_shape=in_shape) ,
      LeakyReLU(alpha=0.2) ,

      Conv2D(filters=128 ,kernel_size=(3,3) ,padding='same' ,strides=(2,2)) ,
      LeakyReLU(alpha=0.2) ,

      Conv2D(filters=128 ,kernel_size=(3,3) ,padding='same' ,strides=(2,2)) ,
      LeakyReLU(alpha=0.2) ,

      Conv2D(filters=256 ,kernel_size=(3,3) ,padding='same' ,strides=(2,2)) ,
      LeakyReLU(alpha=0.2) ,

      Flatten() ,
      Dropout(0.4) ,
      Dense(units=1 ,activation='sigmoid')
  ])

  optimizer = Adam(learning_rate=0.0002 ,beta_1=0.5)
  model.compile(loss='binary_crossentropy' ,optimizer=optimizer ,metrics=['accuracy'])

  return model

In [ ]:
# model = define_descriminator()
# model.summary()

In [ ]:
# plot_model(model=model ,to_file='descriminator.png' ,show_shapes=True)

In [ ]:
def load_real_samples():

  (xtrain,_) ,(_,_) = load_data()
  X = xtrain.astype('float32')
  X = (X - 127.5) / 127.5
  return X

In [ ]:
# X = load_real_samples()
# X.shape

In [ ]:
# X[0]

In [ ]:
def generate_real_samples(dataset, n_samples):
    half_batch = int(n_samples / 2)
    ix = np.random.randint(0, dataset.shape[0], half_batch)
    X = dataset[ix]
    y = np.ones((half_batch, 1))
    return X, y

In [ ]:
# X ,y = generate_real_samples(X ,64)
# X.shape

In [ ]:
def generate_fake_samples(n_samples):
    size = int(32 * 32 * 3 * n_samples)
    X = np.random.rand(size)
    X = -1 + X * 2
    X = X.reshape((n_samples, 32, 32, 3))
    y = np.zeros((n_samples, 1))
    return X, y

In [ ]:
X ,y = generate_fake_samples(64)
X.shape

In [ ]:
def train_descriminator(model ,dataset ,n_iter=20 ,n_batch=128):
  half_batch = n_batch / 2
  for i in range(n_iter):
    x_real ,y_real = generate_real_samples(dataset ,half_batch)
    _ ,train_acc = model.train_on_batch(x_real ,y_real)

    x_fake ,y_fake = generate_fake_samples(int(half_batch))
    _ ,test_acc = model.train_on_batch(x_fake ,y_fake)

    print(f'real : {train_acc*100}%  fake : {test_acc*100}%')

In [ ]:
model = define_descriminator()
dataset = load_real_samples()
# dataset
train_descriminator(model ,dataset)

Generator

In [ ]:
from keras.models import Sequential
from keras.layers import Dense ,Reshape ,Conv2D ,Conv2DTranspose ,LeakyReLU
from keras.utils import plot_model

In [ ]:
# defining Generator model
def define_generator(latent_dim):
  n_nodes = 256 *4 *4
  model = Sequential([
      #foundation for 4x4 image
      Dense(units=n_nodes ,input_dim=latent_dim) ,
      LeakyReLU(alpha=0.2) ,
      Reshape((4 ,4 ,256)) ,

      #upsample to 8x8
      Conv2DTranspose(filters=128 ,kernel_size=(4,4) ,strides=(2,2) ,padding='same') ,
      LeakyReLU(alpha=0.2) ,

      #upsample to 16x16
      Conv2DTranspose(filters=128 ,kernel_size=(4,4) ,strides=(2,2) ,padding='same') ,
      LeakyReLU(alpha=0.2) ,

      #upsample to 32x32
      Conv2DTranspose(filters=128 ,kernel_size=(4,4) ,strides=(2,2) ,padding='same') ,
      LeakyReLU(alpha=0.2) ,

      Conv2D(filters=3 ,kernel_size=(3,3) ,activation='relu' ,padding='same')
  ])
  return model

In [ ]:
latent_dim = 100
gmodel = define_generator(latent_dim)
gmodel.summary()

In [ ]:
plot_model(model=gmodel ,to_file='generator.png' ,show_shapes=True)

In [ ]:
def generate_latent_points(latent_dim ,n_samples):
  x_input = np.random.randn(latent_dim * n_samples)
  x_input = x_input.reshape(n_samples ,latent_dim)
  return x_input

In [ ]:
x_input = generate_latent_points(latent_dim=100 ,n_samples=64)
x_input.shape

In [ ]:
def generate_fake_samples(gmodel ,latent_dim ,n_samples):
  x_input = generate_latent_points(latent_dim ,n_samples)
  X = gmodel.predict(x_input)
  y = np.zeros((n_samples ,1))
  return X ,y

In [ ]:
latent_dim = 100
gmodel = define_generator(latent_dim)

n_samples = 25

X ,_ = generate_fake_samples(gmodel ,latent_dim ,n_samples)
X = (X + 1) / 2.0

for i in range(n_samples):
  plt.subplot(5 ,5 ,i+1)
  plt.axis('off')
  plt.imshow(X[i])
plt.show()

In [ ]:
def define_gan(g_model ,d_model):
  d_model.trainable = False

  model = Sequential([
      g_model ,
      d_model
  ])
  opt = Adam(learning_rate=0.0002 ,beta_1=0.5)
  model.compile(optimizer=opt ,loss='binary_crossentropy' ,metrics=['accuracy'])

  return model

In [ ]:
latent_dim = 100

d_model = define_descriminator()
g_model = define_generator(latent_dim)

gan_model = define_gan(g_model ,d_model)

gan_model.summary()

plot_model(gan_model ,to_file='gan.png' ,show_shapes=True ,show_layer_names=True)

In [ ]:
# train the Generator and Descriminator
def train_gan(g_model ,d_model ,gan_model ,dataset ,latent_dim ,n_epochs=200 ,n_batch=128):
  batch_per_epoch = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  for i in range(n_epochs):
    for j in range(batch_per_epoch):
      X_real , y_real = generate_real_samples(dataset ,half_batch)
      d_loss1 , _ = d_model.train_on_batch(X_real ,y_real)

      X_fake , y_fake = generate_fake_samples(g_model ,latent_dim ,half_batch)
      d_loss2 , _ = d_model.train_on_batch(X_fake ,y_fake)

      X_gan = generate_latent_points(latent_dim ,n_batch)
      y_gan = np.ones((n_batch ,1))

      g_loss = gan_model.train_on_batch(X_gan ,y_gan)

      print(f'>{i+1}, {j+1}/{batch_per_epoch}, d1={d_loss1} ,d2={d_loss2} ,g={g_loss}')

    if (i+1) % 10 == 0:
      summarize_performance(i ,g_model ,d_model ,dataset ,latent_dim)

In [ ]:
# Evaluate the Descriminator ,plot generated images ,save generator model
def summarize_performance(i ,g_model ,d_model ,dataset ,latent_dim):
  X_real , y_real = generate_real_samples(dataset ,n_samples)
  _ , acc_real = d_model.evaluate(X_real ,y_real ,verbose=0)

  X_fake , y_fake = generate_fake_samples(g_model ,latent_dim ,n_samples)
  _ , acc_fake = d_model.evaluate(X_fake ,y_fake ,verbose=0)

  print(f'>Accuracy -->  real:{acc_real*100}% fake:{acc_fake*100}%')

  save_plot(X_fake ,epoch=i)

  filename = f'generator_model_{i+1}.h5'
  g_model.save(filename)

In [ ]:
def save_plot(examples ,epoch ,n=5):
  examples = (examples + 1) / 2.0

  for i in range(n*n):
    plt.subplot(n ,n ,i+1)
    plt.axis('off')
    plt.imshow(examples[i])
  filename = f'generator_model_{epoch+1}.png'
  plt.savefig(filename)
  plt.close()

In [ ]:
pwd

In [ ]:
cd GAN

In [ ]:
pwd

In [ ]:
train_gan(g_model ,d_model ,gan_model ,dataset ,latent_dim ,n_epochs=200 ,n_batch=128)